In [2]:
import numpy as np
import pandas
from random import random
from copy import deepcopy


/tmp/ipykernel_102278/2865488907.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [3]:
A = np.array([[1.8,-0.77],[1,0]])
B = np.array([[1],
              [0]])
C = np.array([1,0.8])
gama = 0.97
alpha_c = 0.1
lam = 0.65

In [4]:
import torch 
import torch.nn as nn 
from torch.autograd.functional import jacobian
loss_fn = nn.MSELoss()

def model(i,o):
    model = nn.Sequential(
    nn.Linear(i, 10),
    nn.Tanh(),
    nn.Linear(10, o)
    )
    return model

def DNN(func,optimizer, n,x_sample,y_sample,xk, require_jack = False):
    x_sample = np.array(x_sample)
    # func = model(x_sample.shape[1],o)
    # optimizer = torch.optim.Adam(func.parameters(), lr=0.01)
    # loss_fn = nn.MSELoss()
    dists = np.linalg.norm(x_sample-xk,axis=1)
    K = min(n,len(dists))
    nearest_idx = np.argsort(dists)[:K]
    x_mask = x_sample[nearest_idx]
    y_mask = np.array(y_sample)[nearest_idx]
    X = torch.tensor(x_mask).float()
    Y = torch.tensor(y_mask).float()
    for epoch in range(500):
        pred = func(X)
        loss = loss_fn(pred, Y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    grad = return_gradient(func,xk,require_jac=require_jack)
    return grad, nearest_idx

def return_gradient(model, xk, require_jac = False):
   
    x_test = torch.tensor(xk).float().requires_grad_(True)
    y_target = model(x_test)
    x_test.grad = None
    model.zero_grad()
    gradient = torch.zeros(2)
    if not require_jac:
        y_target.backward()
        return x_test.grad
    else:
        for i in range(2):
            x_test.grad = None
            model.zero_grad()
            y_target[i].backward(retain_graph = True)
            gradient[i] = x_test.grad[2]
        return gradient
            
def saturate(u, limit=5.0):
    return np.clip(u, -limit, limit)

        
    
    

In [5]:

aa = 0.01  # reduced actor learning rate
alpha_c = 0.01  # reduced critic learning rate
gama = 0.1
lam = 0.8

xa_sample, ya_sample = [], []
xp_sample, yp_sample = [], []
xc_sample, yc_sample = [], []
elig_trace = dict()

# xk = np.array([1, 1])
xk = np.array([1,1])
uk = np.array([random()])

xp_sample.append(np.concatenate((xk, uk)))
xa_sample.append(xk)
xc_sample.append(xk)

ya_sample.append(uk.item())
yc_sample.append(0.0)
log, log_a, log_c = [], [], []

Wa = np.array([1,1,1,1])
Wp = np.array([1, 1, 1, 1,1])

converge_cnt = 0
converge_cnta = 0
converge_cntc = 0
# B = B.reshape(4,1)

func_a = model(2,1)
opti_a = torch.optim.Adam(func_a.parameters(), lr=0.01)
func_p = model(3,2)
opti_p = torch.optim.Adam(func_p.parameters(), lr=0.01)
func_c = model(2,1)
opti_c = torch.optim.Adam(func_c.parameters(), lr=0.01)

for i in range(500):
    noise = random()
    xk = A @ xk + B @ uk
    if np.linalg.norm(xk) > 15:  
        xk = np.random.rand(2,)
    xk = np.round(xk, decimals=2)

    yp_sample.append(xk)
    rk = -(xk @ xk + uk.item() ** 2)



    # Actor update
    # beta_a, mask_idxA = LLR(xa_sample, ya_sample, xk, 9, Wa)
    alias_a = deepcopy(xk)
    grad_a, mask_idxA = DNN(func_a,opti_a,9,xa_sample,ya_sample,alias_a)
    alias_a = torch.tensor(alias_a).float()
    uk = saturate((func_a(alias_a) + 0.001 * noise).detach().numpy())

    # Policy model
    # beta_p,_= LLR(xp_sample, yp_sample, xp_sample[-1], 9, Wp)
    alias_p = deepcopy(xp_sample[-1])    
    grad_p, _ = DNN(func_p,opti_p,9,xp_sample,yp_sample,alias_p,True)
    xp_sample.append(np.concatenate((xk, uk)))

    # Critic update 
    # beta_c, mask_idxC = LLR(xc_sample, yc_sample, xk, 20, Wa)
    alias_c = deepcopy(xk)
    grad_c, mask_idxC= DNN(func_c,opti_c,20,xc_sample,yc_sample,alias_c)


    for m in mask_idxA:
        try:
            ya_sample[m] = saturate(ya_sample[m] + aa * grad_c.numpy() @ grad_p.numpy()) #use grad_p here
        except Exception:
            pass  

    # Vk = beta_c @ np.concatenate((xk, [1]))
    alias_c = torch.tensor(alias_c).float()
    Vk = func_c(alias_c).detach().numpy()
    delta_k = rk + gama * Vk - yc_sample[-1]
    # print(Vk,delta_k)

    # Update eligibility traces
    for j in mask_idxC:
        key = xc_sample[j].tobytes()
        elig_trace[key] = 1.0
        yc_sample[j] += alpha_c * delta_k * 1.0

    all_idxs = set(range(len(yc_sample)))
    other_idxs = all_idxs - set(mask_idxC)

    for k in other_idxs:
        key = xc_sample[i].tobytes()
        elig_trace[key] = elig_trace.get(key, 0) * gama * lam
        elig_trace[key] = min(elig_trace[key], 5.0) 
        yc_sample[k] += alpha_c * delta_k * elig_trace[key]
    xa_sample.append(xk)
    ya_sample.append(uk.item())
    xc_sample.append(xk)
    yc_sample.append(Vk)
    print("grad_c: ",grad_c, "grad_a: ",grad_a,"grad_p:", grad_p)
  

/home/akshit/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/akshit/.local/lib/python3.10/site-packages/torch/autograd/__init__.py:266: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


grad_c:  tensor([ 0.2084, -0.1166]) grad_a:  tensor([-0.0543,  0.1576]) grad_p: tensor([0.2811, 0.0641])


/home/akshit/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


grad_c:  tensor([ 0.1976, -0.0949]) grad_a:  tensor([-0.0722,  0.1159]) grad_p: tensor([-0.2917, -0.1990])


/home/akshit/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([3])) that is different to the input size (torch.Size([3, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

grad_c:  tensor([0.2884, 0.1747]) grad_a:  tensor([0.0446, 0.1094]) grad_p: tensor([-1.2951e-05, -1.3511e-05])


/home/akshit/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


grad_c:  tensor([0.2287, 0.0776]) grad_a:  tensor([-0.0265,  0.1006]) grad_p: tensor([-1.0171, -0.6808])


/home/akshit/.local/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([9])) that is different to the input size (torch.Size([9, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


grad_c:  tensor([0.1956, 0.1141]) grad_a:  tensor([-0.0501,  0.0735]) grad_p: tensor([-1.1680, -0.7492])
grad_c:  tensor([0.1335, 0.4015]) grad_a:  tensor([-0.0333,  0.0440]) grad_p: tensor([-1.2975, -1.3261])
grad_c:  tensor([-0.1368,  1.0038]) grad_a:  tensor([-0.0210,  0.0248]) grad_p: tensor([-0.6112, -0.9081])
grad_c:  tensor([-1.0051,  2.5338]) grad_a:  tensor([-0.0073,  0.0082]) grad_p: tensor([-1.3460, -1.4872])
grad_c:  tensor([-1.5440,  3.1094]) grad_a:  tensor([-0.0023,  0.0026]) grad_p: tensor([6.0324, 4.3882])
grad_c:  tensor([ 0.1201, -0.1465]) grad_a:  tensor([0.2152, 0.2160]) grad_p: tensor([13.6368, 10.3483])
grad_c:  tensor([ 0.3465, -0.4510]) grad_a:  tensor([ 0.0019, -0.0167]) grad_p: tensor([0.2904, 0.2889])
grad_c:  tensor([ 0.5564, -0.3527]) grad_a:  tensor([-0.0094,  0.0183]) grad_p: tensor([0.8052, 0.4921])
grad_c:  tensor([-0.1188,  0.4801]) grad_a:  tensor([ 0.0025, -0.0018]) grad_p: tensor([0.7630, 0.2541])
grad_c:  tensor([-0.9342,  1.9544]) grad_a:  tensor

In [6]:
func_a.parameters,func_a[0].weight,func_a[0].bias

(<bound method Module.parameters of Sequential(
   (0): Linear(in_features=2, out_features=10, bias=True)
   (1): Tanh()
   (2): Linear(in_features=10, out_features=1, bias=True)
 )>,
 Parameter containing:
 tensor([[-3.1467, -2.4381],
         [-1.7937, -1.0890],
         [ 1.9362,  1.7865],
         [-0.4440, -0.4891],
         [-0.1201,  1.2809],
         [-0.2673,  0.3075],
         [-1.5314,  1.8598],
         [-0.7279, -0.6085],
         [-0.2980,  0.2084],
         [ 2.2343,  2.2345]], requires_grad=True),
 Parameter containing:
 tensor([-5.3943, -3.7022,  1.9518, -0.7023,  2.2056,  0.3548, -1.2008, -0.6030,
          0.6504,  2.1071], requires_grad=True))

In [7]:
func_c.parameters,func_c[0].weight,func_c[0].bias

(<bound method Module.parameters of Sequential(
   (0): Linear(in_features=2, out_features=10, bias=True)
   (1): Tanh()
   (2): Linear(in_features=10, out_features=1, bias=True)
 )>,
 Parameter containing:
 tensor([[ 0.2817, -0.7328],
         [ 3.1496, -7.6767],
         [ 1.0007,  0.8430],
         [-1.1050,  2.3796],
         [ 2.1602, -4.2822],
         [ 2.7334, -4.1337],
         [ 1.1583, -1.0015],
         [-2.9462,  4.2893],
         [ 0.2624,  0.9978],
         [-1.1690,  2.0501]], requires_grad=True),
 Parameter containing:
 tensor([ 12.9791,   8.7935,  -4.5947,   5.0956,  -0.1716,   6.2491, -14.2754,
         -10.0855,   4.5147, -11.8379], requires_grad=True))

In [8]:
func_p.parameters,func_p[0].weight,func_p[0].bias

(<bound method Module.parameters of Sequential(
   (0): Linear(in_features=3, out_features=10, bias=True)
   (1): Tanh()
   (2): Linear(in_features=10, out_features=2, bias=True)
 )>,
 Parameter containing:
 tensor([[ -2.2016,  -2.9492,  -4.6408],
         [  1.5882,  -1.0697,  -1.4058],
         [  2.0962,  -1.4885,   0.7006],
         [ -0.4745,  -0.7656,  -5.4954],
         [ -0.5014,   1.2152,   0.2516],
         [  0.9020,  -0.2884,   3.2327],
         [ -3.5127,  -5.9321,  -5.8208],
         [ -4.2047,   3.4150,   9.5494],
         [  0.5012,   1.1966,   4.3109],
         [ -1.3262,  -0.5320, -10.0855]], requires_grad=True),
 Parameter containing:
 tensor([ -6.0113,  10.2727,  -3.5397,  -6.3362, -15.0920,   6.9607,  -2.5715,
          12.0230,   6.1377,  -3.1943], requires_grad=True))

In [12]:
torch.save(func_a.state_dict(),'weights/actor_param_new.pt')
torch.save(func_c.state_dict(),'weights/critic_params_new.pt')
torch.save(func_p.state_dict(),'weights/process_params_new.pt')